In [1]:
import pandas as pd
import numpy as np
import tensorly as ts
import urllib3
import json

In [2]:
# Dataframe configs
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
# Setting SonarCloud.io project key
#chave = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd:packages&branch=master'

frontend_key = 'Guilherme-Aguiar_2020.1-VC_Gestor-FrontEnd&branch=master'
backend_key = 'Guilherme-Aguiar_2020.1-VC_Gestor-BackEnd&branch=master'

#backend
# https://sonarcloud.io/api/measures/component_tree?component=Guilherme-Aguiar_2020.1-VC_Gestor-BackEnd&branch=master&metricKeys=complexity,functions,files,comment_lines_density,duplicated_lines_density,coverage,sqale_rating,test_success_density,bugs,open_issues,reliability_rating,vulnerabilities,security_rating,ncloc,comment_lines
# frontend
# https://sonarcloud.io/api/measures/component_tree?component=Guilherme-Aguiar_2020.1-VC_Gestor-FrontEnd&branch=master&metricKeys=complexity,functions,files,comment_lines_density,duplicated_lines_density,coverage,sqale_rating,test_success_density,bugs,open_issues,reliability_rating,vulnerabilities,security_rating,ncloc,comment_lines

In [4]:
# Defining the raw data extract from sonar cloud
metrics_list = ['files',
               'functions',
               'complexity',
               'ncloc',
               'comment_lines',
               'comment_lines_density',
               'duplicated_lines_density',
               'test_success_density',
               'coverage',
               'bugs',
               'reliability_rating',
               'vulnerabilities',
               'security_rating',
               'open_issues',
               'sqale_rating']

In [5]:
print(len(metrics_list))

15


In [6]:
# Assembling the metric list string and API request URLs

def generate_metric_string(metric_list):
    metric_str = ''

    for metric in metric_list:
        metric_str += metric + ','

    return metric_str

In [7]:
metric_str = generate_metric_string(metrics_list)

In [8]:
# Defining URLs

frontend_url = f"https://sonarcloud.io/api/measures/component_tree?component={frontend_key}&metricKeys={metric_str}&ps=500"
backend_url = f"https://sonarcloud.io/api/measures/component_tree?component={backend_key}&metricKeys={metric_str}&ps=500"

In [9]:
# Starting connection
http = urllib3.PoolManager()

In [10]:
# Sonar Cloud API Request
frontend_request = http.request('GET', frontend_url)
print(frontend_request.status)

backend_request = http.request('GET', backend_url)
print(backend_request.status)

200
200


In [11]:
# Decoding json files extracted from sonar
frontend_json = json.loads(frontend_request.data.decode('utf-8'))
backend_json = json.loads(backend_request.data.decode('utf-8'))

In [12]:
'''
# Saving json files locally
file_name = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd-R2.json'

with open(file_name, 'w') as outfile:
    json.dump(json_repository, outfile)
'''

def metric_per_file(json):
    file_json = []

    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)

    return file_json

In [13]:
frontend_file_json = metric_per_file(frontend_json)
backend_file_json = metric_per_file(backend_json)

In [14]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)

    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass

    df.reset_index(inplace = True)
    df.drop(['files'], axis=1)
    df = df.rename({'index': 'path'}, axis=1)

    return df

In [15]:
frontend_files_data = generate_file_dataframe(metrics_list, frontend_file_json, language_extension = 'js')
backend_files_data = generate_file_dataframe(metrics_list, backend_file_json, language_extension = 'js')

In [16]:
# FRONTEND METRICS DATA PER FILE

frontend_files_data

,path,files,functions,complexity,ncloc,comment_lines,comment_lines_density,duplicated_lines_density,test_success_density,coverage,bugs,reliability_rating,vulnerabilities,security_rating,open_issues,sqale_rating
0,src/services/api.js,1,0,0,5,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
1,src/components/app/App.js,1,1,1,25,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
2,src/components/app/App.test.js,1,1,1,8,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
3,src/components/pages/BeneficiosCriar.js,1,1,1,20,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,1,1.0
4,src/components/component-name/component-name.js,1,0,0,0,0,NaN,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
5,src/components/components/Forms.js,1,4,10,99,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,3,1.0
6,src/components/pages/Postagem.js,1,3,3,53,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
7,src/components/components/PostagemMenu.js,1,10,13,62,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
8,src/components/components/Sidebar.js,1,8,12,109,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,3,1.0
9,src/components/pages/tabela.js,1,3,3,66,5,7.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0


In [17]:
# BACKEND METRICS DATA PER FILE

backend_files_data


,path,files,functions,complexity,ncloc,comment_lines,comment_lines_density,duplicated_lines_density,test_success_density,coverage,bugs,reliability_rating,vulnerabilities,security_rating,open_issues,sqale_rating
0,packages/benefits/src/models/benefit.js,1,0,0,34,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
1,packages/benefits/src/controller/BenefitContro...,1,7,7,69,1,1.4,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
2,packages/resolution/src/models/category.js,1,0,0,21,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
3,packages/resolution/src/test/controller/Contro...,1,12,14,90,2,2.2,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
4,packages/benefits/src/config/database.js,1,0,0,10,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
5,packages/resolution/src/config/database.js,1,0,0,10,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
6,packages/user/index.js,1,1,1,11,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
7,packages/resolution/src/test/controller/ModelT...,1,7,7,40,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
8,packages/resolution/src/models/place.js,1,0,0,24,0,0.0,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0
9,packages/resolution/src/models/post.js,1,1,1,83,11,11.7,0.0,100.0,NaN,0,1.0,0,1.0,0,1.0


In [18]:
#printing CC per file descriptive statistics
backend_files_data['complexity'].astype(int).describe()

count    14.000000
mean      2.928571
std       4.304993
min       0.000000
25%       0.000000
50%       0.500000
75%       6.250000
max      14.000000
Name: complexity, dtype: float64

In [19]:
backend_files_data['complexity/functions']=backend_files_data['complexity'].astype(float)/backend_files_data['functions'].astype(float)

#printing the average CC per function descriptive statistics
backend_files_data['complexity/functions'].describe()

count    7.000000
mean     1.214286
std      0.497347
min      1.000000
25%      1.000000
50%      1.000000
75%      1.083333
max      2.333333
Name: complexity/functions, dtype: float64

In [20]:
# Defining metrics for measure em1, where em1 = density of non-complex file
m0 = np.median(backend_files_data['complexity/functions'])
m1 = backend_files_data['complexity'].astype(float)
m2 = backend_files_data['functions'].astype(float)
m3 = len(backend_files_data)

#  intervals for em1 interpretation
x = np.array([0, m0])
y = np.array([1, 0])

# Defining intervals for em1 interpretation
IF1 = np.interp(list(((m1/m2) < m0)[(m2 > 0)]),x, y)

em1 = sum(IF1)/m3

em1

0.5

In [21]:
# Defining metrics for measure em2, where em2 = density of commented files
m4 = backend_files_data['comment_lines_density'].astype(float)

#  intervals for em1 interpretation
x = np.array([0.1, 0.3])
y = np.array([1, 0])

#(m4 >= 0.1 and m4 <= 0.3)

# Defining intervals for em2 interpretation
em2i = []
for if2i in m4:
    if 10 <= if2i <= 30:
        em2i.append(np.interp(if2i/100,x, y))
    else:
        em2i.append(0)

em2 = np.sum(em2i)/m3

em2

0.06535714285714286

In [22]:
# Defining metrics for measure ma3,where ma3 = absence of duplication on file (density)
m5 = backend_files_data['duplicated_lines_density'].astype(float)

#  intervals for em1 interpretation
x = np.array([0.1, 0.2])
y = np.array([1, 0])

# Intervals for ma3 interpretation
em3i = []
for if3i in m5:
    if 10 <= if3i <= 20:
        em3i.append(np.interp(if3i/100,x, y))
    else:
        em3i.append(0)

em3 = np.sum(em3i)/m3

em3

0.0

In [23]:
# Defining  Modifiability quality sub-characteristic tensor.
# wem1 = 33%
# wem2 = 33%
# wem3 = 33%

wem1=wem2=wem3 = 0.3374

SC_Modifiability_EM_Weights = np.array([wem1, wem2, wem3]).reshape(1,3,1)
SC_Modifiability_EM = np.array([em1, em2, em3]).reshape(1,3,1)


T_SC_Modifiability = ts.tensor(SC_Modifiability_EM)


print("Modifiability Tensor" + '\n' + str(T_SC_Modifiability))
print(T_SC_Modifiability.ndim)

Modifiability Tensor
[[[0.5       ]
  [0.06535714]
  [0.        ]]]
3


In [24]:
# Weighting Modifiability tensor at measure level.

for i in range(T_SC_Modifiability.ndim):
     T_SC_Modifiability[::T_SC_Modifiability.ndim][0][i-1] = \
         ts.kron(T_SC_Modifiability[::T_SC_Modifiability.ndim][0][i-1] ,
                 SC_Modifiability_EM_Weights[::T_SC_Modifiability.ndim][0][i-1])

print("Weighted SC_Modifiability Tensor measures" + '\n' + str(T_SC_Modifiability))

Weighted SC_Modifiability Tensor measures
[[[0.1687   ]
  [0.0220515]
  [0.       ]]]


In [25]:
# Weighting Modifiability tensor at sub-characteristic level.
# wsc1 = 100%

wsc1 = 1

T_SC_Modifiability = ts.kron(T_SC_Modifiability,wsc1)

print("Weighted SC_Modifiability Tensor" + '\n' + str(T_SC_Modifiability))

Weighted SC_Modifiability Tensor
[[[0.1687   ]
  [0.0220515]
  [0.       ]]]


In [26]:
# Defining the maintainability quality characteristic tensor.

T_C_Maintainability = np.array([])
T_C_Maintainability = np.append(T_C_Maintainability,T_SC_Modifiability)

# Weighting Maintainability tensor at characteristic level.
# wc1 = 100%

wc1 = 1

T_C_Maintainability = ts.kron(T_C_Maintainability,wc1)

# Weighted C_Maintainability Tensor
print(str(T_C_Maintainability.reshape(1,3,1)))

[[[0.1687   ]
  [0.0220515]
  [0.       ]]]


In [27]:
# Aggregating C_Maintainability Tensor. The aggregation view is obtained from Frobenius norm of tensor.
print(str(ts.norm(T_C_Maintainability)))

0.1701351188092864


In [28]:
SQC_1 = np.array([])
SQC_1 = np.append(SQC_1, T_C_Maintainability)

# The overall quality of Release 1 of the VC_Gestor system.
# Github Repository: https://github.com/
print(str(ts.norm(SQC_1)))


0.1701351188092864
